<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [17]:
def date_time(table_cells):
    """
    This function returns the date and time from the HTML table cell.
    Input: the element of a table data cell extracts extra.
    """
    # Uses a list comprehension to strip whitespace from each string found in the table_cells
    # and collects them into a list. The result is sliced to get the first two elements.
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell.
    Input: the element of a table data cell extracts extra row.
    """
    # Creates a new string by joining every other element (even indexed) in table_cells.strings,
    # excluding the last element, by using list comprehension and the join() method.
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out  # Returns the constructed booster version string

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell.
    Input: the element of a table data cell extracts extra row.
    """
    # Takes the first element in table_cells.strings and assigns it to 'out'.
    out = [i for i in table_cells.strings][0]
    return out  # Returns the landing status

def get_mass(table_cells):
    # Normalizes the text content of table_cells, converting it to a standard encoding and stripping whitespace.
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    
    # Checks if mass is not an empty string
    if mass:
        # Finds the index where "kg" appears in the string
        mass.find("kg")  
        # Slices the string to include everything up to and including "kg"
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = 0  # If there's no mass, assigns 0
    
    return new_mass  # Returns the new mass value

def extract_column_from_header(row):
    """
    This function returns the column name from the HTML table header cell.
    Input: the element of a table data cell extracts extra row.
    """
    # Checks and extracts any <br> elements from the row (removes them from the structure)
    if row.br:
        row.br.extract()
    # Checks and extracts any <a> elements (removes them from the structure)
    if row.a:
        row.a.extract()
    # Checks and extracts any <sup> elements (removes them from the structure)
    if row.sup:
        row.sup.extract()
    
    # Joins all remaining contents of the row into a single string, which represents the column name
    column_name = row.get_text(separator=' ', strip=True)
    
    # Filters out strings that are entirely digits or empty names
    if not (column_name.strip().isdigit()):
        column_name = column_name.strip()  # Strips whitespace from the column name
        return column_name  # Returns the cleaned column name


In [4]:
# To Define a User-Agent string that identifies your intent without needing a domain as wiki bot policy 
# was blocking webscraping request/response. This is like politely knocking the door before entering to scrape!
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; StudyScraper/1.0)'
}
robots_url = 'https://en.wikipedia.org/robots.txt'
robots_txt = requests.get(robots_url, headers=headers).text

# If you want to read the Wikipedia bot policy
# print(robots_txt)


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
# assign the response to a object
data = requests.get(static_url, headers=headers).text

# This request include the User-Agent identifier string (same as above) so that it knows who you are (bot policy)

In [7]:
# print(data)

Create a `BeautifulSoup` object from the HTML `response`


In [8]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(data, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 

In [9]:
# Use soup.title attribute
tag_object=soup.title
print("tag object:",tag_object)

tag object: <title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [21]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

# Loop through each table, extract and list header names
for i, table in enumerate(html_tables):
    headers = []
    for th in table.find_all('th'):
        cleaned_name = extract_column_from_header(th)
        if cleaned_name:  # Skip None or empty strings
            headers.append(cleaned_name)
    print(f"Table {i+1} headers: {headers}")

Table 1 headers: []
Table 2 headers: []
Table 3 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Table 4 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Table 5 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Table 6 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'N/A']
Table 7 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Table 8 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Table 9 headers: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'FH 2', 'FH 3']

Starting from the third table is our target table contains the actual launch records.


In [22]:
# Let's print the third table and check its content
Third_table = html_tables[2]
# print(Third_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [24]:
#column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

# Function to extract column names from header elements
# def extract_column_from_header(th_element):
    # return th_element.get_text(strip=True)  # Using strip to clean up the text

column_names = []  # List to hold non-empty column names

# Find all <th> elements within the third table
headers = html_tables[2].find_all('th')

# Iterate over each header
for th in headers:
    name = extract_column_from_header(th)  # Extract the column name
    # Check if the name is not None and has length greater than 0
    if name is not None and len(name) > 0:
        column_names.append(name)  # Append non-empty names to the list



Check the extracted column names


In [25]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [27]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']
# del launch_dict['Version,Booster[b]']
# del launch_dict['Boosterlanding']
# del launch_dict['Payload[c]']
# del launch_dict['Launchoutcome']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [28]:
# Set a counter to track how many valid launch rows have been processed.
extracted_row = 0
# You should always initialize counters before loops

# Loop through all launch tables on the page that match a specific CSS class
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Passing a string like "wikitable plainrowheaders collapsible" as the second argument is interpreted as 
    # a filter for the class_ attribute, but it’s not robust. It requires an exact match of the entire class string, 
    # including order and spacing
    
    for rows in table.find_all("tr"):
    # Iterate through each <tr> table row in the current table line by line
        
        # Check for flight number (a digit in the header cell) in the header cell to identify valid launch entries
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()  # Only process if it's a number
        else:
            flag = False

        row = rows.find_all('td')
        # Get all <td> cells in the row — the actual data

        # Only process rows that look like valid launch data (i.e. with enough columns and a valid flight number.)
        if flag and len(row) >= 9:
            extracted_row += 1

            # --- Date and Time ---
            datatimelist = date_time(row[0]) if row[0] else []
            # Date value
            date = datatimelist[0].strip(',') if len(datatimelist) > 0 else None # grabs the first item in the list, strip() removes the comma
            # Time value
            time = datatimelist[1] if len(datatimelist) > 1 else None # grabs the 2nd item in the list

            # --- Booster Version ---
            bv = booster_version(row[1]) if row[1] else None
            if not bv and row[1] and row[1].a and row[1].a.string:
               bv = row[1].a.string.strip()

            # bv = booster_version(row[1]) if row[1] else None
            # if not bv:
                # bv = row[1].get_text(strip=True) if row[1] else None
            # print(bv)


            # --- Launch Site ---
            launch_site = row[2].a.string.strip() if row[2] and row[2].a and row[2].a.string else (
                row[2].get_text(strip=True) if row[2] else None)

            # --- Payload ---
            payload = row[3].a.string.strip() if row[3] and row[3].a and row[3].a.string else (
                row[3].get_text(strip=True) if row[3] else None)

            # --- Payload Mass ---
            payload_mass = get_mass(row[4]) if row[4] else 'N/A'

            # --- Orbit ---
            orbit = row[5].a.string.strip() if row[5] and row[5].a and row[5].a.string else (
                row[5].get_text(strip=True) if row[5] else None)

            # --- Customer ---
            customer = row[6].a.string.strip() if row[6] and row[6].a and row[6].a.string else (
                row[6].get_text(strip=True) if row[6] else None)
            

            # --- Launch Outcome ---
            try:
                launch_outcome = list(row[7].strings)[0].strip() if row[7] else 'Unknown'
            except (IndexError, AttributeError):
                launch_outcome = 'Unknown'

            # --- Booster Landing ---
            booster_landing = landing_status(row[8]) if row[8] else 'Unknown'

            # Clean unwanted annotations like [1], [citation needed] from text
            def clean_text(text):
                return re.sub(r"\[.*?\]", "", text) if isinstance(text, str) else text

            # --- Append clean values to the launch dictionary ---
            launch_dict['Flight No.'].append(clean_text(flight_number))
            launch_dict['Date'].append(clean_text(date))
            launch_dict['Time'].append(clean_text(time))
            launch_dict['Version Booster'].append(clean_text(bv))
            launch_dict['Launch site'].append(clean_text(launch_site))
            launch_dict['Payload'].append(clean_text(payload))
            launch_dict['Payload mass'].append(clean_text(payload_mass))
            launch_dict['Orbit'].append(clean_text(orbit))
            launch_dict['Customer'].append(clean_text(customer))
            launch_dict['Launch outcome'].append(clean_text(launch_outcome))
            launch_dict['Booster landing'].append(clean_text(booster_landing))


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [29]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [30]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [42]:
df.to_csv('spacex_web_scraped.csv', index=False)


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
